In [179]:
import pandas as pd
df = pd.read_csv('test_files/testcsv.csv')
df.head()

,a,b,c
0,1,2,3


In [180]:
import camelot
tables = camelot.read_pdf('test_files/transactions.pdf', flavor='stream', pages="1-end")
tables

<TableList n=2>

In [181]:
camelot_df = pd.concat([table.df for table in tables], ignore_index=True)
camelot_df

,0,1,2,3,4
0,,,,Transaction history,NaN
1,,,,Date: 12/18/2025,NaN
2,50 transactions,,,,NaN
3,Search:from:2025-09-24 to:2025-12-14,,,,NaN
4,DATE,BUSINESS WEBSITE OR DESCRIPTION,DEBITS,CREDITS,NaN
...,...,...,...,...,...
57,10/28/2025,PAYMENT VENMO 3264681992,,-$11.80,
58,10/28/2025,PAYMENT VENMO 3264681992,,-$10.00,
59,10/28/2025,PAYMENT VENMO 3264681992,,-$8.64,
60,10/28/2025,PAYMENT VENMO 3264681992,,-$8.48,


In [182]:
# # Function to detect and convert date-like values
# def convert_dates_in_series(series):
#     # Try converting each cell individually
#     def try_parse(val):
#         if pd.isna(val):
#             return val
#         try:
#             parsed = pd.to_datetime(val, errors='raise')
#             return parsed
#         except Exception:
#             return val  # Leave unchanged if not a date
#     return series.apply(try_parse)

# # Apply to entire DataFrame
# converted_camelot_df = camelot_df.apply(convert_dates_in_series)

"""
Pandas operations become a lot simpler when we transform for our specific bank statement format.
Assuming the first column is always the date column, we can directly convert it.
"""
converted_camelot_df = camelot_df.copy()
converted_camelot_df[0] = pd.to_datetime(converted_camelot_df[0], errors='coerce')  # Convert first column to datetime
converted_camelot_df.dropna(subset=[0], inplace=True,ignore_index=True)  # Drop rows where date conversion failed
converted_camelot_df.head()

C:\Users\caywo\AppData\Local\Temp\ipykernel_27908\1263227278.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  converted_camelot_df[0] = pd.to_datetime(converted_camelot_df[0], errors='coerce')  # Convert first column to datetime


,0,1,2,3,4
0,2025-12-11,ACH PMT AMEX EPAYMENT 0005000008 M2176 0910000...,"-$2,081.02",,NaN
1,2025-12-11,CAYWOOD MITCHELLD 1411681137MN DEPT OF DEED 04...,,"$1,828.00",NaN
2,2025-12-08,BILL PAID-DOUG CAYWOOD CONF #1 7,-$41.92,,NaN
3,2025-12-08,PAYMENT VENMO 3264681992,"-$1,496.75",,NaN
4,2025-12-08,TRANSFER AMERICAN EXPRESS XXXX5066 12/08/25 CA...,,$41.92,NaN


In [183]:
#Convert last three columns to numeric
def currency_to_numeric(series):
    # Remove $ and commas, strip spaces
    cleaned = series.astype(str).str.replace(r'[\$,]', '', regex=True).str.strip()
    # Convert to numeric, invalid parsing becomes NaN
    return pd.to_numeric(cleaned, errors='coerce')

# Apply conversion
for col in converted_camelot_df.columns[2:]:
    converted_camelot_df[col] = currency_to_numeric(converted_camelot_df[col]).fillna(0)

converted_camelot_df.head()

,0,1,2,3,4
0,2025-12-11,ACH PMT AMEX EPAYMENT 0005000008 M2176 0910000...,-2081.02,0.00,0.0
1,2025-12-11,CAYWOOD MITCHELLD 1411681137MN DEPT OF DEED 04...,0.00,1828.00,0.0
2,2025-12-08,BILL PAID-DOUG CAYWOOD CONF #1 7,-41.92,0.00,0.0
3,2025-12-08,PAYMENT VENMO 3264681992,-1496.75,0.00,0.0
4,2025-12-08,TRANSFER AMERICAN EXPRESS XXXX5066 12/08/25 CA...,0.00,41.92,0.0


In [184]:
#sum last three columns then drop them
converted_camelot_df['amount'] = converted_camelot_df.iloc[:, 2:].sum(axis=1)
converted_camelot_df.drop(converted_camelot_df.columns[[2,3,4]], axis=1, inplace=True)
converted_camelot_df.head()

,0,1,amount
0,2025-12-11,ACH PMT AMEX EPAYMENT 0005000008 M2176 0910000...,-2081.02
1,2025-12-11,CAYWOOD MITCHELLD 1411681137MN DEPT OF DEED 04...,1828.00
2,2025-12-08,BILL PAID-DOUG CAYWOOD CONF #1 7,-41.92
3,2025-12-08,PAYMENT VENMO 3264681992,-1496.75
4,2025-12-08,TRANSFER AMERICAN EXPRESS XXXX5066 12/08/25 CA...,41.92


In [185]:
#rename columns and convert to json-like format
converted_camelot_df.columns = ['date', 'description', 'amount']
converted_camelot_df.to_dict(orient='records')

[{'date': Timestamp('2025-12-11 00:00:00'),
  'description': 'ACH PMT AMEX EPAYMENT 0005000008 M2176 091000017244122 ER AM',
  'amount': -2081.02},
 {'date': Timestamp('2025-12-11 00:00:00'),
  'description': 'CAYWOOD MITCHELLD 1411681137MN DEPT OF DEED 042000014427855 UI BENEFIT',
  'amount': 1828.0},
 {'date': Timestamp('2025-12-08 00:00:00'),
  'description': 'BILL PAID-DOUG CAYWOOD CONF #1 7',
  'amount': -41.92},
 {'date': Timestamp('2025-12-08 00:00:00'),
  'description': 'PAYMENT VENMO 3264681992',
  'amount': -1496.75},
 {'date': Timestamp('2025-12-08 00:00:00'),
  'description': 'TRANSFER AMERICAN EXPRESS XXXX5066 12/08/25 CAYWOOD,MITCHELXXXX7623 PERSONAL',
  'amount': 41.92},
 {'date': Timestamp('2025-12-02 00:00:00'),
  'description': 'TRANSFER AMERICAN EXPRESS XXXX5066 12/02/25 CAYWOOD,MITCHELXXXX2034 PERSONAL',
  'amount': 167.0},
 {'date': Timestamp('2025-12-01 00:00:00'),
  'description': 'CAYWOOD MITCHELLD 1411681137MN DEPT OF DEED 042000010827648 UI BENEFIT',
  'amount

In [186]:

# import pdfplumber
# with pdfplumber.open("test_files/transactions.pdf") as pdf:
#     for line in pdf.pages[0].extract_text().splitlines():
#         print("NEW LINE:", line)


In [187]:

# import tabula
# df = tabula.read_pdf("test_files/transactions.pdf", pages='all')
# for i in range(5):
#     print(df[i])


In [188]:
# from openai import OpenAI
# from dotenv import load_dotenv
# import os

# load_dotenv()
# api_key = os.getenv("OPENAI_API_KEY")
# if not api_key:
#     raise ValueError("OPENAI_API_KEY environment variable is not set.")

# client = OpenAI(api_key=api_key)

# file = client.files.create(
#     file=open("test_files/transactions.pdf","rb"),
#     purpose='user_data'
# )

# response = client.responses.create(
#     model="gpt-5",
#     input=[
#         {"role": "user",
#         "content": [
#             {"type": "input_file",
#             "file_id": file.id
#         },
#         {
#             "type": "input_text",
#             "text": "Extract all transactions from the bank statement in a JSON array with date, description, amount, and balance."
#         },
#         ]
#         }
#         ]
# )

# print(response)

In [189]:
# print(response.output[1].content[0].text)